In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_earnings_report_df(ticker):
  cols = [
    'TotalRevenue',                   # Top-line growth
    'OperatingRevenue',              # Recurring core revenue
    'NetIncome',                     # Bottom-line profitability
    'NetIncomeCommonStockholders',  # Tied directly to EPS
    'DilutedNIAvailtoComStockholders', # Accurate EPS calc
    'DilutedEPS', 'BasicEPS',        # Core valuation metrics
    'DilutedAverageShares', 'BasicAverageShares',  # EPS denominator
    'PretaxIncome', 'TaxProvision',  # Profit before/after tax
    'NormalizedIncome',              # Adjusted earnings
    'NetIncomeContinuousOperations', # Core operations performance
]

  df_q = yf.Ticker(ticker) \
    .get_income_stmt(freq='quarterly') \
    .T[cols]
  df_q = df_q.sort_index()
  df_q.insert(0, 'EndDate', df_q.index)
  df_q.insert(0, 'StartDate', df_q.index.to_series().shift(1) + pd.Timedelta(days=1))
  df_q.insert(0, 'Granularity', 'Quarterly')

  df_y = yf.Ticker(ticker) \
    .get_income_stmt(freq='yearly') \
    .T[cols]
  df_y = df_y.sort_index()
  df_y.insert(0, 'EndDate', df_y.index)
  df_y.insert(0, 'StartDate', df_y.index.to_series().shift(1) + pd.Timedelta(days=1))
  df_y.insert(0, 'Granularity', 'Yearly')

  df = pd.concat([df_q, df_y])
  df.insert(0, 'Ticker', ticker)
  df.reset_index(drop=True, inplace=True)

  return df

In [ ]:
def get_balance_sheet_df(ticker):
  cols = [
    'TotalDebt',                        # Leverage
    'TangibleBookValue',                # Asset-based valuation
    'InvestedCapital',                  # Capital base
    'NetTangibleAssets',                # Net assets
    'CommonStockEquity',                # Equity base
    'StockholdersEquity',               # Total equity
    'RetainedEarnings',                 # Cumulative profits
    'CapitalStock',                     # Capital structure
    'CommonStock',                      # Shareholder equity
    'TotalLiabilitiesNetMinorityInterest', # Total liabilities
    'PayablesAndAccruedExpenses',       # Short-term liabilities
    'TotalAssets',                      # Total size of the firm
    'NetPPE',                           # Property, plant, and equipment
    'CashAndCashEquivalents'            # Liquidity position
  ]
  df_q = yf.Ticker(ticker) \
    .get_balance_sheet(freq='quarterly') \
    .T[cols]
  df_q = df_q.sort_index()
  df_q.insert(0, 'EndDate', df_q.index)
  df_q.insert(0, 'StartDate', df_q.index.to_series().shift(1) + pd.Timedelta(days=1))
  df_q.insert(0, 'Granularity', 'Quarterly')

  df_y = yf.Ticker(ticker) \
    .get_balance_sheet(freq='yearly') \
    .T[cols]
  df_y = df_y.sort_index()
  df_y.insert(0, 'EndDate', df_y.index)
  df_y.insert(0, 'StartDate', df_y.index.to_series().shift(1) + pd.Timedelta(days=1))
  df_y.insert(0, 'Granularity', 'Yearly')

  df = pd.concat([df_q, df_y])
  df.insert(0, 'Ticker', ticker)
  df.reset_index(drop=True, inplace=True)

  return df

In [ ]:
def add_df_cols(df, final_df):
  granularity_map = {
    'Quarterly': '_q',
    'Yearly': '_y'
  }

  for col in df.columns:
    if col != 'EndDate' and col != 'StartDate' and col != 'Granularity' and col != 'Ticker':
      final_df[col+'_q'] = pd.NA
      final_df[col+'_y'] = pd.NA

  for row in df.itertuples():
      start = row.StartDate.tz_localize('America/New_York')
      end = row.EndDate.tz_localize('America/New_York')
      granularity = row.Granularity

      for col in df.columns:
        if col != 'EndDate' and col != 'StartDate' and col != 'Granularity' and col != 'Ticker':
          final_df.loc[
            (final_df.index >= start) & (final_df.index <= end),
            f'{col}{granularity_map[granularity]}'
          ] = getattr(row, col)

In [ ]:
def get_econ_data(min_date, max_date):
  min_date = min_date - pd.Timedelta(weeks=6)
  max_date = max_date + pd.Timedelta(weeks=6)

  # Interest Rates
  ffr = pdr.DataReader('FEDFUNDS', 'fred', min_date, max_date)
  ffr.columns = ['FederalFundsRate']
  prime = pdr.DataReader('PRIME', 'fred', min_date, max_date)
  prime.columns = ['PrimeRate']
  gs10 = pdr.DataReader('GS10', 'fred', min_date, max_date)
  gs10.columns = ['10YrTreasuryRate']

  # Inflation
  cpi = pdr.DataReader('CPIAUCSL', 'fred', min_date, max_date)
  cpi.columns = ['CPI']
  core_cpi = pdr.DataReader('CPILFESL', 'fred', min_date, max_date)
  core_cpi.columns = ['CoreCPI']

  # Economic Output
  gdp = pdr.DataReader('GDP', 'fred', min_date, max_date)
  gdp.columns = ['GDP']
  industrial_production = pdr.DataReader('INDPRO', 'fred', min_date, max_date)
  industrial_production.columns = ['IndustrialProduction']

  # Labor Market
  unemployment_rate = pdr.DataReader('UNRATE', 'fred', min_date, max_date)
  unemployment_rate.columns = ['UnemploymentRate']
  nonfarm_payrolls = pdr.DataReader('PAYEMS', 'fred', min_date, max_date)
  nonfarm_payrolls.columns = ['NonfarmPayments']
  avg_hrly_earnings = pdr.DataReader('CES0500000003', 'fred', min_date, max_date)
  avg_hrly_earnings.columns = ['AvgHourlyEarnings']

  ##Corporate Data
  corp_profits_after_tax = pdr.DataReader('CP', 'fred', min_date, max_date)
  corp_profits_after_tax.columns = ['CorporateProfitsAfterTax']

  #Credit Conditions
  consumer_credit = pdr.DataReader('CCLACBW027SBOG', 'fred', min_date, max_date)
  consumer_credit.columns = ['ConsumerLoans']
  bank_lending_data = pdr.DataReader('TOTLL', 'fred', min_date, max_date)
  bank_lending_data.columns = ['BankLoansAndLeases']

  #Stock Market Data
  sp500 = pdr.DataReader('SP500', 'fred', min_date, max_date)
  sp500.columns = ['SP500Index']

  #Exchange Rates
  us_dollar_index = pdr.DataReader('DTWEXBGS', 'fred', min_date, max_date)
  us_dollar_index.columns = ['USDIndex']

  econ_data = [
      ffr,
      prime,
      gs10,
      cpi,
      core_cpi,
      gdp,
      industrial_production,
      unemployment_rate,
      nonfarm_payrolls,
      avg_hrly_earnings,
      corp_profits_after_tax,
      consumer_credit,
      bank_lending_data,
      sp500,
      us_dollar_index
    ]

  econ_df = pd.DataFrame()

  for df in econ_data:
    df.index = df.index.tz_localize('America/New_York')
    econ_df = econ_df.merge(df, left_index=True, right_index=True, how='outer')
    econ_df.ffill(inplace=True)

  return econ_df

In [ ]:
ticker = 'AAPL'

earnings_df = get_earnings_report_df(ticker)
balance_sheet_df = get_balance_sheet_df(ticker)

min_date = earnings_df.StartDate.min()
max_date = earnings_df.EndDate.max()

final_df = yf.Ticker(ticker).history(start=min_date, end=max_date)
econ_df = get_econ_data(min_date, max_date)

add_df_cols(earnings_df, final_df)
add_df_cols(balance_sheet_df, final_df)

final_df = final_df.merge(econ_df, left_index=True, right_index=True, how='left')
final_df.insert(1, 'Sector', yf.Ticker('AAPL').info['sector'])

final_df.head()

,Open,Sector,High,Low,Close,Volume,Dividends,Stock Splits,TotalRevenue_q,TotalRevenue_y,...,GDP,IndustrialProduction,UnemploymentRate,NonfarmPayments,AvgHourlyEarnings,CorporateProfitsAfterTax,ConsumerLoans,BankLoansAndLeases,SP500Index,USDIndex
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00-04:00,114.710600,Technology,114.788609,112.945674,113.881767,116120400,0.0,0.0,<NA>,365817000000.0,...,22068.767,96.5256,6.9,142461.0,29.55,2398.03,753.2677,10488.8084,3380.80,116.7339
2020-10-02 00:00:00-04:00,110.078905,Technology,112.497154,109.425591,110.205666,144712000,0.0,0.0,<NA>,365817000000.0,...,22068.767,96.5256,6.9,142461.0,29.55,2398.03,753.2677,10488.8084,3348.44,116.7424
2020-10-05 00:00:00-04:00,111.073512,Technology,113.745281,110.722476,113.599014,106243800,0.0,0.0,<NA>,365817000000.0,...,22068.767,96.5256,6.9,142461.0,29.55,2398.03,753.2677,10488.8084,3408.63,116.1870
2020-10-06 00:00:00-04:00,112.818938,Technology,113.228485,109.454850,110.342194,161498200,0.0,0.0,<NA>,365817000000.0,...,22068.767,96.5256,6.9,142461.0,29.55,2398.03,753.2677,10488.8084,3360.95,116.2781
2020-10-07 00:00:00-04:00,111.765819,Technology,112.672661,111.288015,112.214363,96849000,0.0,0.0,<NA>,365817000000.0,...,22068.767,96.5256,6.9,142461.0,29.55,2398.03,751.2546,10466.2922,3419.45,116.3548


In [ ]:
# 1. Market Leaders (FAANG / Magnificent 7)
market_leaders = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA']

# 2. Sector Representation
technology = ['AAPL', 'MSFT', 'NVDA']
healthcare = ['JNJ', 'PFE', 'UNH']
financials = ['JPM', 'BAC', 'GS']
consumer_discretionary = ['AMZN', 'HD', 'TSLA']
consumer_staples = ['PG', 'KO', 'WMT']
energy = ['XOM', 'CVX']
industrials = ['CAT', 'GE']
utilities = ['NEE', 'DUK']
real_estate = ['AMT', 'PLD']
materials = ['LIN', 'NEM']
telecom = ['VZ', 'T']

# 3. Indices & ETFs
indices_etfs = ['SPY', 'QQQ', 'DIA', 'IWM']

# 4. High Volatility / Retail Favorites
high_volatility = ['GME', 'AMC', 'PLTR']

# 5. Defensive / Safe-Haven Stocks
defensive = ['JNJ', 'KO', 'WMT', 'MCD']

# Combine all unique tickers into one list
all_tickers = list(set(
    market_leaders +
    technology + healthcare + financials + consumer_discretionary +
    consumer_staples + energy + industrials + utilities + real_estate +
    materials + telecom +
    high_volatility +
    defensive
))

print(all_tickers)

['AMC', 'LIN', 'KO', 'JPM', 'MSFT', 'PG', 'GOOGL', 'NVDA', 'GME', 'AMT', 'BAC', 'JNJ', 'XOM', 'GE', 'HD', 'CAT', 'PFE', 'DUK', 'CVX', 'META', 'TSLA', 'VZ', 'T', 'NEE', 'NEM', 'UNH', 'WMT', 'PLD', 'PLTR', 'MCD', 'AAPL', 'GS', 'AMZN']


In [ ]:
df_all = pd.DataFrame()

for ticker in all_tickers:
  print(f"Getting info for {ticker}")

  earnings_df = get_earnings_report_df(ticker)
  balance_sheet_df = get_balance_sheet_df(ticker)

  min_date = earnings_df.StartDate.min()
  max_date = earnings_df.EndDate.max()

  final_df = yf.Ticker(ticker).history(start=min_date, end=max_date)
  econ_df = get_econ_data(min_date, max_date)

  add_df_cols(earnings_df, final_df)
  add_df_cols(balance_sheet_df, final_df)

  final_df = final_df.merge(econ_df, left_index=True, right_index=True, how='left')
  final_df.insert(0, 'Ticker', ticker)
  final_df.insert(1, 'Sector', yf.Ticker(ticker).info['sector'])

  df_all = pd.concat([df_all, final_df])

Getting info for AMC
Getting info for LIN
Getting info for KO
Getting info for JPM
Getting info for MSFT
Getting info for PG
Getting info for GOOGL
Getting info for NVDA
Getting info for GME
Getting info for AMT
Getting info for BAC
Getting info for JNJ
Getting info for XOM
Getting info for GE
Getting info for HD
Getting info for CAT
Getting info for PFE
Getting info for DUK
Getting info for CVX
Getting info for META
Getting info for TSLA
Getting info for VZ
Getting info for T
Getting info for NEE
Getting info for NEM
Getting info for UNH
Getting info for WMT
Getting info for PLD
Getting info for PLTR
Getting info for MCD
Getting info for AAPL
Getting info for GS
Getting info for AMZN


In [ ]:
df_all.shape

(31943, 78)

In [ ]:
df_all.head()

,Ticker,Sector,Open,High,Low,Close,Volume,Dividends,Stock Splits,TotalRevenue_q,...,GDP,IndustrialProduction,UnemploymentRate,NonfarmPayments,AvgHourlyEarnings,CorporateProfitsAfterTax,ConsumerLoans,BankLoansAndLeases,SP500Index,USDIndex
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,AMC,Communication Services,22.000000,22.000000,20.000000,20.100000,2987380,0.0,0.0,<NA>,...,22656.793,98.8135,6.4,142913.0,29.92,2716.546,747.3243,10340.3132,3700.65,111.2143
2021-01-05 00:00:00-05:00,AMC,Communication Services,19.900000,20.299999,19.100000,19.799999,2814830,0.0,0.0,<NA>,...,22656.793,98.8135,6.4,142913.0,29.92,2716.546,747.3243,10340.3132,3726.86,110.9649
2021-01-06 00:00:00-05:00,AMC,Communication Services,20.299999,22.299999,19.700001,20.100000,6736330,0.0,0.0,<NA>,...,22656.793,98.8135,6.4,142913.0,29.92,2716.546,747.4342,10337.4329,3748.14,110.8573
2021-01-07 00:00:00-05:00,AMC,Communication Services,20.799999,21.100000,20.200001,20.500000,2615050,0.0,0.0,<NA>,...,22656.793,98.8135,6.4,142913.0,29.92,2716.546,747.4342,10337.4329,3803.79,111.3124
2021-01-08 00:00:00-05:00,AMC,Communication Services,20.900000,22.100000,20.700001,21.400000,3955330,0.0,0.0,<NA>,...,22656.793,98.8135,6.4,142913.0,29.92,2716.546,747.4342,10337.4329,3824.68,111.2930


In [ ]:
df_all.to_csv("/content/drive/MyDrive/ISAN Project/Datasets/data_all.csv", index=True)

In [ ]:
econ_df.to_csv("/content/drive/MyDrive/ISAN Project/Datasets/economics.csv", index=True)